# 深層学習入門(ニューラルネットワーク系) - Chapter3

- **[3.1 Kerasの便利な機能](#3.1-Kerasの便利な機能)**
    - **[3.1.1 学習過程の表示](#3.1.1-学習過程の表示)**
    - **[3.1.2 EarlyStopping](#3.1.2-EarlyStopping)**
    - **[3.1.3 モデル構造の出力](#3.1.3-モデル構造の出力)**
<br><br>
- **[3.2 精度向上](#3.2-精度向上)**
    - **[3.2.1 最適化](#3.2.1-最適化)**
    - **[3.2.2 ドロップアウト](#3.2.2-ドロップアウト)**
    - **[3.2.3 ファインチューニング](#3.2.3-ファインチューニング)**

***

## 3.1 Kerasの便利な機能

### 3.1.1 学習過程の表示

Kerasでは学習時のモデルの精度の遷移などを記録することができます。  
これをグラフとして表示することで、精度がどのように変化したかを視覚的に確認することができます。
以下のようにして、学習時の履歴を変数に保存できます。
```python
history = model.fit(X_train, y_train, batch_size=32, epoch=5)
```
これで得られた履歴から、精度の変化を得るには以下のようにします。
```python
history.history['acc']
```
また、以下のように`fit()`にテストデータを渡すことでそのデータを汎化精度の計算に用います。
```python
history = model.fit(X_train, y_train, batch_size=32, epoch=5, validation_data=(X_test, y_test))
```
この、汎化精度の履歴は下のようにして得られます。
```python
history.history['val_acc']
```

#### 問題

以下のプログラムを埋めて学習時の精度(accとval_acc)の変化をプロットするプログラムを完成させてください。

In [ ]:
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense
from keras.models import Sequential, load_model
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:10000]
X_test = X_test.reshape(X_test.shape[0], 784)[:1000]
y_train = to_categorical(y_train)[:10000]
y_test = to_categorical(y_test)[:1000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation('sigmoid'))
model.add(Dense(128))
model.add(Activation('sigmoid'))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(optimizer='sgd', loss='categorical_crossentropy',
              metrics=['accuracy'])
# historyの取得

#acc, val_accのプロット

plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc='best')
plt.show()

#### 解答例

In [ ]:
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Activation, Dense
from keras.models import Sequential, load_model
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784)[:10000]
X_test = X_test.reshape(X_test.shape[0], 784)[:1000]
y_train = to_categorical(y_train)[:10000]
y_test = to_categorical(y_test)[:1000]

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Activation('sigmoid'))
model.add(Dense(128))
model.add(Activation('sigmoid'))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(optimizer='sgd', loss='categorical_crossentropy',
              metrics=['accuracy'])
# historyの取得
history = model.fit(X_train, y_train, batch_size=32,
                    epochs=10, validation_data=(X_test, y_test))

#acc, val_accのプロット
plt.plot(history.history['acc'], label='acc', ls='-', marker='o')
plt.plot(history.history['val_acc'], label='val_acc', ls='-', marker='x')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc='best')
plt.show()

### 3.1.2 Early Stopping

ニューラルネットワークの学習を行う際、学習が進んでいくと、減少していたテストデータに対する損失が増加し始めることがあります。  
このような場合は、早めに学習を打ち切ったほうが汎化精度が高くなります。  
それを行う方法がEarly Stopping(早期終了)です。
早期終了とは、過学習を防ぐために途中で学習を打ち切ることです。<br>
学習用・テスト用データとは別に過学習の検証用データを持っておき、学習毎に学習用データと検証用データの評価誤差を監視し、誤差が広がった場合、学習をストップする。<br>
Kerasでは学習時のcallback関数に指定することでEarly Stoppingが行えます。
callbackは以下のように指定します。
```python
from keras.callbacks import EarlyStopping
#Early Stoppingの定義
es = EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='min')
#学習時のcallbacksに指定
model.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=[es])
```
`monitor`は、監視する値で、`patience`は監視する値が何エポック改善しなかったら打ち切るかを指定する引数です。  
`mode`は、モデルの改善というのが、監視する値が減少する場合か、増加する場合かを指定する引数で、`min`のとき指定する値の減少がモデルの改善を意味し、`max`のとき指定する値の増加がモデルの改善を意味します。  
これを`auto`と指定することで、`monitor`に指定された値から自動で推定します。

### 3.1.3 モデル構造の出力

Kerasにはモデルの構造を画像として出力する関数が用意されています。  
これを用いることで自分が作成したモデルの構造を視覚的に認識でき、人に伝えるときにも分かりやすくなります。
構造の出力は以下のように行います。
```python
from keras.utils import plot_model
plot_model(model, to_file='model.png')
```
これにより、modelの構造がカレントディレクトリにmodel.pngとして保存されます。
このときに、表示される名称は、各層を追加する際に下記のように、`name`で指定できます。
```python
model.add(Dense(10, name='output_layer'))
```
指定しない場合、「層の名称_番号」のようになります。 (例えば、3つ目の全結合層であればdense_3となります)

#### 問題

プログラムを埋めて、層の構造を出力してください。  
ただし、層の名称は、"dense_256", "dense_128", "dense_10", "softmax"としてください。

In [ ]:
import matplotlib.pyplot as plt
from keras.layers import Activation, Dense
from keras.models import Sequential
from keras.utils.vis_utils import plot_model

% matplotlib inline

model = Sequential()
model.add(Dense(256, input_dim=784))
model.add(Dense(128))
model.add(Dense(10))
model.add(Activation('softmax', name="softmax"))

# モデル構造を保存

image = plt.imread('model.png')
plt.figure(figsize=(2, 2.8), dpi=200)
plt.xticks([])
plt.yticks([])
plt.imshow(image)
plt.show()

#### 解答例

In [ ]:
import matplotlib.pyplot as plt
from keras.layers import Activation, Dense
from keras.models import Sequential
from keras.utils.vis_utils import plot_model

% matplotlib inline

model = Sequential()
model.add(Dense(256, input_dim=784, name="dense_256"))
model.add(Dense(128, name="dense_128"))
model.add(Dense(10, name="dense_10"))
model.add(Activation('softmax', name="softmax"))

# モデル構造を保存
plot_model(model, 'model.png')

image = plt.imread('model.png')
plt.figure(figsize=(2, 2.8), dpi=200)
plt.xticks([])
plt.yticks([])
plt.imshow(image)
plt.show()

## 3.2 精度向上

### 3.2.1 最適化

学習率は、学習が進むにつれて変化するほうがうまく学習できる場合が多いです。  
これを学習率の最適化といい、Adam、RMSprop、Adagradなど様々な最適化方法が提案されてきました。  
この中で最もよく使われるのはAdamなので、最初はAdamを用いて学習し、精度が高くならない場合などに他の方法に変更すると良いです。  

### 3.2.2 ドロップアウト

ニューラルネットは複数のモデルを組み合わせることで、精度が向上します。<br>
しかし、ニューラルネットワークの構造が複雑化していくにつれて、ニューロンの重みは訓練データセットに最適化されていってしまいます。``複数のモデルをそれぞれ学習させる``のには、**計算資源や時間がたくさん必要**になります。<br>
そこで、1つのモデルのユニットの一部を学習のたびに``ランダムに除去``することで、これに似た結果を得ようとするのが**ドロップアウト**です。<br>
これにより、ニューラルネットは特定のニューロンの存在に依存できなくなり、より``汎用的``な(学習データのみに依存しない)特徴を学習するようになります。<br>
その結果、学習データに対する**オーバーフィッティング(過学習)**を防ぐことができます。

### 3.2.3 ファインチューニング

大規模なニューラルネットを学習させるのにはとても時間がかかり、データも大量に必要になります。  
このような場合は、大量のデータですでに学習されているモデルの一部を用いて学習させます。
例えば、画像認識であれば、最後の全結合層のみを変更し、畳み込み部分はそのまま使います。
Kerasでは、Xception、VGG16、VGG19、ResNet50、InceptionV3の5つの画像分類モデルをImageNetで学習した重みをダウンロードし、使用できます。
各モデルは、入力の最低サイズがあり、[このページ](https://keras.io/ja/applications)に載っています。 
ここではVGG16を例に説明します。
まず、VGGのモデルを作ります。
```python
from keras.applications.vgg16 import VGG16

input_tensor = Input(shape=(28, 28, 1))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)
```
input_tersorとして入力の形を与えます。  
include_topは、もとのモデルの全結合層を用いるかどうかです。これをfalseにすることで、特徴抽出部分のみを用いて、それ以降のモデルは自分で作成したモデルと結合することができます。 weightsは'imagenet'を指定すると、imagenetで学習した重みを用い、Noneとするとランダムな重みを用いるようになります。
特徴抽出部分以降に結合するには以下のようにします。
```python
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(10, activation='softmax'))
#入力はvgg.input, 出力は, top_modelにvgg16の出力を入れたもの
model = Model(inputs=vgg16.input, outputs=top_model(vgg16.output))
```
vgg16による特徴抽出部分の重みは更新されると崩れてしまうので以下のようにして固定します。
```python
# modelの15層目までがvggのモデル
for layer in model.layers[:15]:
    layer.trainable = False
```
コンパイル・学習は同様に行えますが、ファインチューニングする場合、最適化はSGDで行うと良いです。
```python
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
```

#### 問題

プログラムを埋めて、cifar10をvgg16のモデルをファインチューニングし学習させてください。（精度は60%ほどにとどまります。訓練データをかさ増ししたり、何度も学習を繰り返せば90%くらい出るそうですが今回それを試す必要はありません）

In [ ]:
from keras import optimizers
from keras.applications.vgg16 import VGG16
from keras.datasets import cifar10
from keras.layers import Dense, Dropout, Flatten, Input
from keras.models import Model, Sequential
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#input_tensorの定義
#----------------------------

#----------------------------

vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(256, activation='sigmoid'))
top_model.add(Dropout(0.5))
top_model.add(Dense(10, activation='softmax'))

# vgg16とtop_modelを連結
#----------------------------

#----------------------------

# 15層目までの重みを固定
#----------------------------


#----------------------------

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print(score)

#### 解答例

In [ ]:
from keras import optimizers
from keras.applications.vgg16 import VGG16
from keras.datasets import cifar10
from keras.layers import Dense, Dropout, Flatten, Input
from keras.models import Model, Sequential
from keras.utils.np_utils import to_categorical

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#input_tensorの定義
input_tensor = Input(shape=(32, 32, 3))

vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(256, activation='sigmoid'))
top_model.add(Dropout(0.5))
top_model.add(Dense(10, activation='softmax'))

# vgg16とtop_modelを連結
model = Model(input=vgg16.input, output=top_model(vgg16.output))

# 15層目までの重みを固定
for layer in model.layers[:15]:
    layer.trainable = False

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print(score)